# Maltese Cuisine Food Recipes 
---

In [1]:
# Libraries needed to get the html from a site and to parse the html
from bs4 import BeautifulSoup as bs
import requests
from parse_ingredients import parse_ingredient
import pandas
import pyfood as pyf

## Base URL
This is the main URL or home page of the Maltese Cuisine site. It is being accessed, parsed and stored in the soup variable.

In [2]:
# Saving the url to a variable, getting the url and parsing the html
mainURL = "https://maltesecuisine.com/"
req = requests.get(mainURL)
soup = bs(req.text, "html.parser")

SSLError: HTTPSConnectionPool(host='maltesecuisine.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1129)')))

## getPages
This function will go through the main URL and the next pages. These pages all contain hrefs to the recipes on this site. It will save the links of these hrefs to the the hrefs list variable that is being returned

In [ ]:
def getPages():
    hrefs = []
    for i in range(1, 12):
        if i == 1:
            req = requests.get("https://maltesecuisine.com")
            soup = bs(req.text, "html.parser")
            temp = soup.findAll("div", class_="post-img")
            for j in temp:
                allAs = j.findAll("a")
                for a in allAs:
                    hrefs.append(a.get('href'))
        else:
            siteURL = "https://maltesecuisine.com/page/" + str(i)
            req = requests.get(siteURL)
            soup = bs(req.text, "html.parser")
            temp = soup.findAll("div", class_="post-img")
            for j in temp:
                allAs = j.findAll("a")
                for a in allAs:
                    hrefs.append(a.get('href'))
    return hrefs

In [ ]:
hrefs = getPages()

## Save recipe links to CSV
These chunks of code will create a CSV file, ListOfRecipes which will have two columns. The one of the left being the recipe's name and the one on the left will store the link to the recipe.

To get the recipe's name, it is using the list of links/hrefs, accessing and parsing the page. Then it finds the h1 tag in the html wihch should be the dish name.

In [10]:
RecipeList = []

for recipe in hrefs:
    req = requests.get(recipe) # Accesses the next recipe
    soup = bs(req.text, "html.parser")
    title = soup.findAll('h1') # finds the name of the recipe

    row = []

    for j in title:
        row.append(str(j.get_text(strip=False)))

    row.append(recipe)

    RecipeList.append(row)

In [17]:
linkList = []
dishList = []

for entry in RecipeList:
    linkList.append(entry[1])
    dishList.append(entry[0])

In [19]:
listOfRecipesCSV = pandas.read_csv('../CSV/listOfRecipes.csv')

In [18]:
recipeListDF = pandas.DataFrame(list(zip(linkList, dishList)), columns=['Links', 'Recipe Name'])

In [20]:
listOfRecipesCSV = listOfRecipesCSV.append(recipeListDF, ignore_index = True)

In [22]:
listOfRecipesCSV.to_csv('../CSV/ListOfRecipes.csv')

## Append ingredients to CSV file
The CSV file created before is being used to get all the Maltese Cuisine dishes and their links.

The getListOfIngredients function will go through a recipe's page whose URL is passed as a parameter. The list of ingredients is being returned. This function does not parse the ingredients insted it takes the entire string found on the page, meaning that if one of the recipe ingredients were '2 breaded veal chops' it will return '2 breaded veal chops'.

The parseIngredientsAndPyfoodFunc function will parse the ingredient passed as a parameter. This function will parse the ingredient and return the parsed ingredient back. Meaning that the example used before, '2 breaded veal chops' will return 'veal' instead.

In [3]:
listOfRecipesCSV = pandas.read_csv('../CSV/listOfRecipes.csv')

In [4]:
def getListOfIngredients(URL):
    ingredients = []
    req = requests.get(URL)
    soup = bs(req.text, "html.parser")
    temp = soup.findAll("div", class_="recipe-ingredients")
    for ul in temp:
        t = ul.findAll("li")
        for ing in t:
            strippedIng = str(ing.get_text(strip=False))
            ingredients.append(strippedIng)
    return ingredients

In [5]:
def parseIngredientsAndPyfoodFunc(ing):
    shelf = pyf.Shelf(region='Italy', month_id=0)
    
    parseResult = parse_ingredient(ing)

    results = shelf.process_ingredients([parseResult.name])
    try:
        pIng = results['ingredients'][0]['foodname']
    except:
        try:
            pIng = results['HS'][0]
        except:
            pIng = parseResult.name

    return pIng

In [7]:
mcLinkList = []
mcDishList = []
mcIngDishList = []
mcParsedIngList = []
counter = 0

for row in listOfRecipesCSV.iterrows():
    temp = []
    if 'maltesecuisine' in row[1][1]:
        mcLinkList.append(row[1][1])
        mcDishList.append(row[1][2])
        mcIngDishList.append(getListOfIngredients(row[1][1]))
        if counter == 0:
            for dish in mcIngDishList:
                for ing in dish:
                    temp.append(parseIngredientsAndPyfoodFunc(ing))
            counter += 1
        else:
            for ing in mcIngDishList[-1]:
                temp.append(parseIngredientsAndPyfoodFunc(ing))
        mcParsedIngList.append(temp)
    else:
        continue
    del temp

C:\Users\maxma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\maxma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
def parseIngredientsAndPyfoodFunc(i, title, shelf):
    data = []
    for j in title:
                data.append(str(j.get_text(strip=False))) # Dish Name
    
    parseResult = parse_ingredient(str(i.get_text(strip=False)))

    results = shelf.process_ingredients([parseResult.name])
    try:
        temp = results['ingredients'][0]['foodname']
    except:
        temp = results['HS'][0]

    temp = temp.replace(" ", "_")

    data.append(temp)

    return data

In [ ]:
ingredientList = []
shelf = pyf.Shelf(region='Italy', month_id=0)

for recipe in linkList:
    req = requests.get(recipe) # Accesses the next recipe
    soup = bs(req.text, "html.parser")
    title = soup.findAll('h1') # finds the name of the recipe
    
    ingredientsDiv = soup.findAll('div', class_='field-name-field-ingredients') # finds all the divs containing the ingredients
    
    # For each div go trough and extract the ingredients
    for ul in ingredientsDiv:
        ingredients = ul.findAll("li")
        
        for i in ingredients:
            # data = parseIngredientsFunc(i, title)
            # data = pyfoodFunc(title, i)
            data = parseIngredientsAndPyfoodFunc(i, title, shelf)
            
            ingredientList.append(data)

In [10]:
df = pandas.DataFrame(columns = ["Recipe", "Ingredient"], data = mcParsedIngList)

df

ValueError: 2 columns passed, passed data had 42 columns

## Finding each ingredients type and index

In [ ]:
ing_info = pandas.read_csv("../CSV/Compound CSVs/ingr_info.tsv", sep="\t")

category = [None] * len(mcParsedIngList)
index = [None] * len(mcParsedIngList)

for ing in range(0, len(mcParsedIngList)):
    for ing2 in range(0, len(ing_info["ingredient name"])):
        if(mcParsedIngList[ing][1] == ing_info["ingredient name"][ing2]):
            category[ing] = ing_info["category"][ing2]
            index[ing] = ing_info["# id"][ing2]
            continue